In [2]:
import cv2
import numpy as np

In [21]:
kernel = np.ones((3, 3), np.uint8)

def Open(GrayImage, erode, dilate):
    erode_img=cv2.erode(GrayImage, kernel, iterations = erode)
    img_open=cv2.dilate(GrayImage, kernel, iterations = dilate)
    return img_open

def Close(GrayImage, erode, dilate):
    dilate_img=cv2.dilate(GrayImage, kernel, iterations = dilate)
    img_close=cv2.erode(GrayImage, kernel, iterations = erode)
    return img_close

In [54]:
cap = cv2.VideoCapture('./Good_horizon_2.mp4')    # Open the specified video file.

fps = cap.get(cv2.CAP_PROP_FPS)
frame_delay = int(1000 / fps)

clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

while(cap.isOpened()):
    ret, frame = cap.read()    # ret -> return value.
    if not ret:
        print("Video is end.")
        break

    frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
    

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    clahe_image = clahe.apply(frame_gray)
    ret,frame_thre = cv2.threshold(clahe_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    frame_thre_close=Close(frame_thre,5,3)
    
    frame_contours,_ = cv2.findContours(frame_thre_close, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    for contours in frame_contours:
        area = cv2.contourArea(contours)
        if 15*10000<area<20*10000:
            cv2.drawContours(frame, contours, -1, (0, 255, 0), 4)

    cv2.imshow('Frame', frame)

    key = cv2.waitKey(frame_delay)
    if key == 27: #Esc
        break

cap.release()
cv2.destroyAllWindows()